# ML4CPS Project-2 | NB-2

In [ ]:
import os
import sys

basepath = os.path.abspath(os.path.join(".."))
if not basepath in sys.path:
    sys.path.append(basepath)

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
%matplotlib inline

In [ ]:
from utils.dataset import DatasetLoaderXL

## Load dataset

In [ ]:
dsxl = DatasetLoaderXL(dataset_dir="../dataset/raw", seed=42, verbose=True)
dsxl.load_all_datasets()

In [ ]:
dsxl.list_suburbs()

In [ ]:
dsxl.list_categories()

In [ ]:
dsxl.list_subcategories("Community")

In [ ]:
dsxl.get_category_across_all_suburbs("Community")
df = dsxl.get_values_for_subcategory_across_all_suburbs()
df